In [ ]:
import os 

import pandas as pd
import geopandas as gpd
from energyemissionsregio.config import DATA_PATH, SHP_PATH
from energyemissionsregio.utils import solve_proxy_equation, get_proxy_var_list
from energyemissionsregio.disaggregation import perform_proxy_based_disaggregation
from energyemissionsregio.plotting_functions import plot_validation_data
from sklearn.metrics import mean_squared_error

In [ ]:
cwd = os.getcwd()

In [ ]:
nuts2_shp = gpd.read_file(os.path.join(SHP_PATH, "NUTS2.shp"))
nuts2_shp = nuts2_shp[nuts2_shp["code"].str.startswith(("DE", "ES"))]

### Eurostat data

In [ ]:
eurostat_data_nuts0 = None
for road_transport_var in ["ghg_emissions_from_fc_in_road_transport_using_light_duty_trucks",
                           "ghg_emissions_from_fc_in_road_transport_using_heavy_duty_trucks_and_buses",
                           "ghg_emissions_from_fc_in_road_transport_using_cars",
                           "ghg_emissions_from_fc_in_road_transport_using_motorcycles"
                           ]:
    data_df = pd.read_csv(os.path.join(DATA_PATH, f"{road_transport_var}.csv"))
    data_df.drop(columns=["year"], inplace=True)

    if eurostat_data_nuts0 is None:
        eurostat_data_nuts0 = data_df
    else:
        eurostat_data_nuts0 = pd.merge(eurostat_data_nuts0, data_df, on="region_code", how="inner")
        eurostat_data_nuts0["value"] = eurostat_data_nuts0["value_x"] + eurostat_data_nuts0["value_y"]
        eurostat_data_nuts0.drop(columns=["value_x", "value_y"], inplace=True)

In [ ]:
eurostat_data_nuts0 = eurostat_data_nuts0[eurostat_data_nuts0["region_code"].str.startswith(("DE", "ES"))][
    ["region_code", "value"]].copy()

In [ ]:
eurostat_data_nuts0

### EDGAR data

In [ ]:
validation_data = pd.read_csv(os.path.join(cwd, "..", "..", "data", 
                                           "validation_data", "transport_emissions_EDGAR.csv"))

validation_data = validation_data[validation_data["code"].str.startswith(("DE", "ES"))][
    ["code", "_sum"]].copy()

validation_data.rename(columns={"code": "region_code", "_sum": "value"}, inplace=True)

validation_data["value"] = validation_data["value"]

In [ ]:
target_data = validation_data.copy()
target_data["region_code"] = target_data["region_code"].str[:2]

target_data = target_data.groupby("region_code").sum().reset_index()

In [ ]:
target_data["value_confidence_level"] = 5

difference between the Eurostat data disaggregated and hotmaps data used for validation of disaggregation 

In [ ]:
diff_df = pd.merge(eurostat_data_nuts0, target_data, on="region_code", suffixes=("_eurostat", "_edgar"))

diff_df["value_edgar"] = diff_df["value_edgar"]/1000000 # Tonnes to Mt

diff_df["diff"] = diff_df["value_eurostat"] - diff_df["value_edgar"]

diff_df

### Disaggregation of EDGAR data

In [ ]:
proxy_equations = {
    "DE": "road_transport_of_freight + \
            (3.83 * de_number_of_passenger_cars_emission_group_euro_1) + \
            (1.78 * de_number_of_passenger_cars_emission_group_euro_2) +\
            (1.25 * de_number_of_passenger_cars_emission_group_euro_3) + \
            (0.825 * de_number_of_passenger_cars_emission_group_euro_4) +\
            (0.735 * de_number_of_passenger_cars_emission_group_euro_5) +\
            (0.6745 * de_number_of_passenger_cars_emission_group_euro_6r) + \
            (0.6745 * de_number_of_passenger_cars_emission_group_euro_6dt) + \
            (0.6745 * de_number_of_passenger_cars_emission_group_euro_6d) +\
            (3.83 * de_number_of_passenger_cars_emission_group_euro_other) + \
            number_of_motorcycles",

  "ES": "road_transport_of_freight + \
        es_average_daily_traffic_light_duty_vehicles + \
        number_of_motorcycles",}

In [ ]:
disagg_data_list = []

for country in ["DE", "ES"]:
    sub_target_data = target_data[target_data["region_code"] == country].copy()

    proxy_equation = proxy_equations[country]

    proxy_var_list = get_proxy_var_list(proxy_equation)

    proxy_data_dict = {}
    for proxy_var in proxy_var_list:
        if os.path.exists(os.path.join(cwd, "..", "..", "data", "disaggregated_data", f"{proxy_var}.csv")):
            proxy_data = pd.read_csv(os.path.join(cwd, "..", "..", "data", "disaggregated_data", f"{proxy_var}.csv"))
        else:
            proxy_data = pd.read_csv(os.path.join(DATA_PATH, f"{proxy_var}.csv"))
        
        proxy_data["value_confidence_level"] = 5

        proxy_data = proxy_data[proxy_data["region_code"].str.startswith(("DE", "ES"))][["region_code", 
                                                                                        "value", 
                                                                                        "value_confidence_level"]].copy()

        proxy_data["value"] = proxy_data["value"].fillna(0)
        proxy_data_dict.update({proxy_var: proxy_data})

    solved_proxy_data = solve_proxy_equation(proxy_equation, proxy_data_dict)

    disagg_data = perform_proxy_based_disaggregation(sub_target_data, solved_proxy_data, "NUTS0", 5)

    disagg_data_list.append(disagg_data)

In [ ]:
disagg_data = pd.concat(disagg_data_list)

In [ ]:
disagg_data["NUTS2"] = disagg_data["region_code"].str[:4]

disagg_data_nuts2 = disagg_data[["NUTS2", "value"]].copy()
disagg_data_nuts2.rename(columns={"NUTS2": "region_code"}, inplace = True)
disagg_data_nuts2 = disagg_data_nuts2.groupby("region_code").sum().reset_index()

In [ ]:
# calulate RMSE and country total -------------
merged_df_mae = pd.merge(validation_data, disagg_data_nuts2, on = "region_code", how="outer", suffixes=("_true", "_disagg"))

true_values_de = merged_df_mae[merged_df_mae["region_code"].str.startswith("DE")]["value_true"]
disagg_values_de = merged_df_mae[merged_df_mae["region_code"].str.startswith("DE")]["value_disagg"]

true_values_es = merged_df_mae[merged_df_mae["region_code"].str.startswith("ES")]["value_true"]
disagg_values_es = merged_df_mae[merged_df_mae["region_code"].str.startswith("ES")]["value_disagg"]

rmse_de = mean_squared_error(true_values_de, disagg_values_de, squared=False).round(2)
rmse_es = mean_squared_error(true_values_es, disagg_values_es, squared=False).round(2)


In [ ]:
rmse_de = "1.53e6"
rmse_es = "2.59e6"

In [ ]:
de_total = "152.99e6"
es_total = "81.09e6"

In [ ]:
fig_path = os.path.join("..", "..", "figures", 
                        "disaggregation_validation", 
                        "validation_road_transport_emissions.png")

plot_validation_data(validation_data, disagg_data_nuts2, 
                     nuts2_shp, de_total, es_total, 
                     rmse_de, rmse_es, "tonnes", "EDGAR", fig_path)